In [ ]:
from surface_code_no_detection_loss_circuit import Rotated_Surface_Code_no_detection_loss
import stim
import numpy as np
import pymatching
import multiprocessing
from multiprocessing import Pool

In [ ]:
#function to compute the logical error rate of the surface code without LDU
def logical_errors(surface_code_circuit, num_shots: int):
    sampler=surface_code_circuit.compile_detector_sampler()
    detection_events,observable_flips=sampler.sample(num_shots,separate_observables=True)
    dem=surface_code_circuit.detector_error_model(decompose_errors=True)
    matcher = pymatching.Matching.from_detector_error_model(dem)
    predictions = matcher.decode_batch(detection_events)
                
    num_errors=0
    for shot in range(num_shots):
           
        predicted_for_shot = predictions[shot]
        actual_for_shot = observable_flips[shot]


        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
                    
    return(num_errors/num_shots)

#Compute the logical error rate of the surface code swithout LDU for a given depolarizing noise probability
#stored as a dictionnary in a file
def simulation_code_surface_code(L,depo,num_shots):
    depo=round(depo,5)
    lattice = Rotated_Surface_Code_no_detection_loss(L,L,L,'|0>','Z',measurement_order='z_first',after_CZ=('DEPOLARIZE2',depo))
    surface_code_circuit=lattice.rotated_surface_code_circuit()
    error_rate=logical_errors(surface_code_circuit,num_shots)

    if os.path.exists('data/logical_error/logical_error_standard_surface_code_d={}.pkl'.format(L)):
        with open('data/logical_error/logical_error_standard_surface_code_d={}.pkl'.format(L), 'rb') as fp:
            data_saved=pickle.load(fp)
    
    else:
        data_saved={}
        
    if depo not in data_saved.keys():#CREATE DICTIONNARY if the depolarizing noises probability has never been computed
        data_saved[depo]={}
        data_saved[depo]['num_shots']=num_shots
        data_saved[depo]['error_rate']=error_rate
    else:#UPDATE DICTIONNARY if there has been already simulation for that given noise probability
        data_saved[depo]['error_rate']*=data_saved[depo]['num_shots']/(data_saved[depo]['num_shots']+num_shots)
        data_saved[depo]['error_rate']+=error_rate*num_shots/(data_saved[depo]['num_shots']+num_shots)
        data_saved[depo]['num_shots']+=num_shots
        
    with open('data/logical_error/logical_error_standard_surface_code_d={}.pkl'.format(L), 'wb') as fp:
        pickle.dump(data_saved, fp)

    print('logical_error for L={}, and strength depo={} is {}'.format(L,depo,error_rate))
    


#wrapper to one argument (needed for parallelization)    
def multi_args_wrapper_surface_code(args):
   simulation_code_surface_code(*args)



In [ ]:
#compute the logical error rate of the standard surface code at vanishing loss probability

Ls = range(11,13,2)#list of distances
depo_list=np.linspace(0.001,0.02,20)#depolarizing noise
num_shots = 1000000#number of shots per core
nb_cores=10 #number of cores
for l,L in enumerate(Ls):
    for d,depo in enumerate(depo_list):
        depo=round(depo,5)
        with Pool(processes=nb_cores) as pool:
            pool.imap(multi_args_wrapper_surface_code,[(L,depo,num_shots) for _ in range(10)])
            pool.close()
            pool.join()